# Use examples of [premise](https://github.com/romainsacchi/premise)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment.

This library extract useful information from IAM model output files (such as those of REMIND or IMAGE) and aligns inventories in the ecoinvent database accordingly.

With version 0.3.7, the following transformation are available:

* `update_electricity()`: create regional electricity markets and adjust efficiency of power plants
* `update_cement()`: creates regional markets for clinker production and adjust clinker production efficiency
* `update_steel()`: creates regional markets for steel and adjust steel production efficiency and the supply of secondary steel
* `update_cars()`: produces fleet average cars and relinks to activities consuming pasenger car trnasport
* `update_trucks()`: produces fleet average trucks and relinks to activities consuming lorry trnasport
* `update_solar_PV()`: updates efficiency of solar PV modules

Additional documentation on the methodology is available [here](https://premise.readthedocs.io/en/latest/introduction.html).

There's also a **pre-print publication** about `premise` [here](https://www.psi.ch/en/ta/preprint).

## Requirements

* **Pyhton 3.9 is highly recommended**
* a user license for ecoinvent v.3
* a **decryption key**, to be asked from [Romain Sacchi](mailto:romain.sacchi@psi.ch)

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an opened `brightway2` project with a biosphere database as well as an ecoinvent v.3 cut-off database registered in that project.

In [ ]:
from premise import *
import brightway2 as bw

In [ ]:
bw.projects

In [ ]:
bw.projects.set_current("my_bw_project")

In [ ]:
list(bw.databases)

### List of available scenarios

Some scenarios come installed with the library.
They are stored in `data/iam_ouput_files` from the root directory.
They are all within the same Shared Socio-Economic Pathway (SSP): SSP2 (nicknamed "middle of the road"), which descrobes a future world (in terms of GDP and demographics development, education, intergovernmental collaboration) very much in line with what has been observed historically..

But they are proposed in combination with different climate mitigation targets, called Representative Concentration Pathways (RCP).
Read more about SSPs and RCPs, [here](https://www.carbonbrief.org/explainer-how-shared-socioeconomic-pathways-explore-future-climate-change).

With REMIND, we have the following SSP/RCP scenarios:
* "SSP2-Base"
* "SSP2-NPi"
* "SSP2-NDC"
* "SSP2-PkBudg1300"
* "SSP2-PkBudg1100"
* "SSP2-PkBudg900"

With IMAGE, we have the following SSP/RCP scenarios:
* "SSP2-Base"
* "SSP2-RCP26"
* "SSP2-RCP19"

For reference:
* the "Base" scenario in REMIND and IMAGE corresponds to RCP 6 (W/m^2), with a temperature increase by 2100 superior to **3.5 C**.
* a RCP of **2.6** (W/m^2) corresponds to the soft target of the Paris Agreement (just **below 2 C** of atmospheric temperature increase by 2100)
* a RCP of **1.9** (W/m^2) corresponds to the ambitious target of the Paris Agreement (**1.5 C** of atmospheric temperature increase by 2100)
* On REMIND's side, "SSP2-Base", "SSP2-PkBudg1300" and "SSP2-PkBudg900" are roughly equivalent in terms of climate mitigation target to "SSP2-Base", "SSP2-RCP26" and "SSP2-RCP19" on IMAGE's side.



### Database creation from default scenarios

To create a scenario using REMIND's SSP2 Base pathway, from ecoinvent 3.6 for the year 2028, one would execute the following cell. This leads to the extraction of the database, some cleanup as well as importing a few additional inventories.

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2029}
            ],
            source_db="ecoinvent 3.5 cutoff", # <-- name of the database in the BW2 project. Must be a string.
            source_version="3.5", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1". Must be a string.
            key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # <-- decryption key
            # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
    )

If you do not want to integrate the IAM projections in the database, but only wish to have the additional inventories, you can stop here and export the database back to Brightway or other destinations, by using the `write_db_to` methods, like so:

In [ ]:
ndb.write_db_to_brightway()

Howver, if you wish first to proceed with the IAM integration, you need to use the `updated_` methods, like so:

In [ ]:
ndb.update_all()

In [ ]:
ndb.write_db_to_brightway()

or here with ecoinvent 3.7.1

In [ ]:
ndb = NewDatabase(
        scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028}
            ],
        source_db="ecoinvent 3.7 cutoff", # <-- this is NEW.
        source_version="3.7.1", # <-- this is NEW
        key='xxxxxxxxxxxxxxxxxxxxxxxxx'
    )

If you want to create multiple databases at once, just populate the `scenarios` list.
You will notice the key `exclude` for which we can list the transformations we do not wish to perform.
In this case, we do not wish to update the electricity sector.

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2020, "exclude": ["update_electricity"]},
                {"model":"remind", "pathway":"SSP2-Base", "year":2030},
                {"model":"remind", "pathway":"SSP2-Base", "year":2040},
                {"model":"remind", "pathway":"SSP2-Base", "year":2050},
            ],
            source_db="ecoinvent 3.7 cutoff", # <-- name of the database. Must be a string.
            source_version="3.7.1", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1"
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
)

In [ ]:
ndb.update_all()

When the database is loaded and the additional inventories imported, you can apply a transformation function.
For example here, we adjust the efficiency of the solar PVs to the two scenarios we have loaded.
We go more in details later.

In [ ]:
ndb.update_solar_PV()

And then, we register these two databases back into brightway2.

In [ ]:
ndb.write_db_to_brightway()

### Database creation from non-default scenarios

If you have some specific IAM scenarios (one that is not included in `premise`) you would like to build a database from, you can specify the directory to those.

**Important remark**: your scenario file must begin with "remind_" or "image_". When using a non-default scenario that you provide yourself, you do not have to provide a decryption key.

In [ ]:
ndb = NewDatabase(
    scenarios = [{"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}],        
    source_db="ecoinvent 3.6 cutoff", # <-- name of the database
    source_version=3.6, # <-- version of ecoinvent
                 )

### Adding inventories
Upon the database extraction, you can import some of your Brightway2-compatible inventories like so:

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2030},
            ],
            source_db="ecoinvent 3.7 cutoff", 
            source_version="3.7.1",
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
            additional_inventories= [ # <-- this is NEW
                {"filepath": r"filepath\to\excel_file.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
                {"filepath": r"filepath\to\another_excel_file.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
            ] # <-- this is NEW
                 )

# Use case with ecospold2

The source database does not have to be from a brightway2 project.
It can be directly extracted from the bunch of ecospold2 files one gets when downloaded from the [ecoinvent website](https://ecoinvent.org).

For this, one needs to specify the argument `source_db = "ecospold"` as well as `source_file_path`, which is the directory leading to the ecospold files.

For example, here we combine the use of a specific (non-default) IAM scenario file with the use of ecospold2 files as data source (ecoinvent 3.5 in this case).

In [ ]:
ndb = NewDatabase(
        scenarios = [
            {"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}
        ],        
        source_type="ecospold", # <--- this is NEW
        source_file_path=r"C:\Users\sacchi_r\Dropbox\Public\ecoinvent 3.5_cutoff_ecoSpold02\datasets", # <-- this is NEW
        source_version="3.5",
    )

# Transformation functions

These functions modify the extracted database:

* **update_electricity()**: alignment of regional electricity production mixes as well as efficiencies for a number of
  electricity production technologies, including Carbon Capture and Storage technologies.
* **update_cement()**: adjustment of technologies for cement production (dry, semi-dry, wet, with pre-heater or not),
  fuel efficiency of kilns, fuel mix of kilns (including biomass and waste fuels) and clinker-to-cement ratio.
* **update_steel()**: adjustment of process efficiency, fuel mix and share of secondary steel in steel markets.
* **update_solar_PV()**: adjustment of solar PV panels efficiency to the year considered.
* **update_cars()**: creates updated inventories for fleet average passenger cars and links back to activities that consume
transport.
* **update_trucks()**: creates updated inventories for fleet average lorry trucks and links back to activities that consume
transport. Such inventories are generated by [carculator](https://github.com/romainsacchi/carculator) for passenger cars and [carculator_truck](https://github.com/romainsacchi/carculator_truck) for medium and heavy-duty trucks.

They can be applied *separately*, *consecutively* or *altogether* (using instead **.update_all()**).
They will apply to all the scenario-specific databases listed in `scenarios`.

In [ ]:
ndb.update_all()

In [ ]:
from premise import *
import brightway2 as bw
bw.projects.set_current("article_carculator")

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {'model':'remind','pathway':'SSP2-Base','year':'2020'},
                {"model":"image", "pathway":"SSP2-Base", "year":2034},
            ],
            key='xxxxxxxxxxxxxxxxxxxxxxxxx',
            source_db="ecoinvent 3.7 cutoff",
            source_version="3.7", 
)

In [ ]:
ndb.update_all()

In [ ]:
ndb.write_db_to_brightway()

You can also give your datababases a custom name.

In [ ]:
ndb.write_db_to_brightway(name=["my_custom_name_1", "my_custom_name_2"])

### Fleet files

A last word about passenger cars and trucks: it is possible to pass a custom fleet composition file to generate fleet average inventories and limit those to specific regions (here, the European region).

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2030,
                 "passenger cars": {"regions":["EUR", "NEU"],
                                    "fleet file":r"filepath/to/fleet_file.csv.csv"},
                 "trucks": {"regions":["EUR"]}
                },
                {"model":"image", "pathway":"SSP2-Base", "year":2030,},
            ],
            key='xxxxxxxxxxxxxxxxxxxxxxxxx',
            source_db="ecoinvent 3.7 cutoff",
            source_version="3.7"
    )

### Exclude specific functions
Finally, we can exclude some transformation functions when executing `update_all()` like so:

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2030,
                 "exclude": ["update_steel"], # <-- do not execute update_seel()
                 "passenger cars": {"regions":["EUR"]},"trucks": {"regions":["EUR"]}
                },
                {"model":"remind", "pathway":"SSP2-Base", "year":2030,},
            ],
            key='xxxxxxxxxxxxxxxxxxxxxxxxx',
            source_db="ecoinvent 3.7 cutoff",
            source_version="3.7", 
)

# Export

### As a Brightway2 database

Export the modified database to brightway2

In [ ]:
ndb.write_db_to_brightway()

### As a sparse matrix representation

Or export it as a sparse matrix representation.

This will export four files:

* "A_matrix.csv": matrix coordinates and values of shape (index of activity; index of product; value) for the technosphere
* "A_matrix_index.csv": labels for indices for A matrix of shape (name of activity, reference product, unit, location, index)
* "B_matrix.csv": matrix coordinates and values of shape (index of activity; index of biosphere flow; value) for the biosphere
* "B_matrix_index.csv": labels for indices for B matrix of shape (name of biosphere flow, main compartment, sub-compartmnet, unit, index)

As a convenience, you can specifiy a directory where to store the exported matrices.
If the directory does not exist, it will be created.
If you leave it unspecified, they will be stored in **data/matrices** in the root folder of the library.

In [ ]:
ndb.write_db_to_matrices(filepath=r"C:/Users/sacchi_r/Downloads/exported_matrices")

### As a SimaPro CSV file

In [ ]:
ndb.write_db_to_simapro(filepath=r"C:/Users/sacchi_r/Downloads/exported_simapro_file")

### As a Superstructure database
A superstructure database is a database that can accomodate several scenarios, as described [here](https://github.com/dgdekoning/brightway-superstructure), to be then used in [Activity-Browser](https://github.com/LCA-ActivityBrowser/activity-browser).
This function will export the superstructure database as well as produce a "scenario difference file". Hence, even though you create multiple scenarios, **you only need to write to disk one database**.

In [ ]:
ndb.write_superstructure_db_to_brightway()